In [ ]:
import importlib
import random
import argparse
import configparser
import numpy as np
import networkx as nx
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_sparse
from torch import Tensor
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.optim as optim

from torch_geometric.utils import negative_sampling, to_networkx
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator


import networkx as nx
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import scipy
import math


from dataset_utils import node_feature_utils
from dataset_utils.node_feature_utils import *
import my_utils as utils

importlib.reload(utils)

In [ ]:
def downsampling(s:np.ndarray, sample_len=64):
    s_len = s.shape[0]
    if s_len <= sample_len:
        downsampled = np.pad(s, pad_width=(0, sample_len-s_len), mode='minimum')
        return np.arange(sample_len), downsampled
    
    indice = np.linspace(0, s_len-1, sample_len)
    int_idc = [int(i) for i in indice]
    downsampled = np.array([s[i] for i in int_idc])
    
    return int_idc, downsampled

x = np.arange(100)/10
sig = x**2 + 0.05*x**1.5

plt.figure()
plt.plot(sig)
int_idc, downsampled = downsampling(sig, sample_len=111)
plt.scatter(int_idc, downsampled, c='r', s=10)
plt.show()

In [ ]:
# plot log(x) <= x/a + log(a) - 1 

plt.figure()
x = np.linspace(0.1, 1, 20)
y = np.log(x)
a = list(np.linspace(0.2, 1, 5))
plt.plot(x, y, label='log x')
for aa in a:
    ya = x/aa + np.log(aa) - 1
    plt.plot(x, ya, label=f'a={aa}')
plt.legend()
plt.show()


In [ ]:
# Conjuagate function:

x = np.linspace(0.01, 2, 20)
fx = x * np.log(x)

y = np.linspace(0.01, 2, 10)


plt.figure()
plt.plot(x, fx)
for yy in y:
    yx = x*yy - fx
    plt.plot(x, yx)
# plt.show()

# plot y:
yy = np.linspace(0.01, 2, 20)

ys = []
for y in yy:
    ys.append(np.max(x*y - fx))
plt.plot(yy, ys)
plt.show()



